In [ ]:
#Import data

In [100]:
from __future__ import print_function  # Python 2 and 3
from sklearn.preprocessing import StandardScaler
import pandas as pd
import scipy.stats
import numpy as np
from numpy.random import shuffle
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LassoCV, ElasticNetCV
import xgboost as xgb
from sklearn.svm import SVR
#from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
plt.style.use('ggplot')
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import decimal
import sklearn.feature_selection as fs
from sklearn import metrics

# Read in Training Set & Filter For Variance

In [42]:
categorical_columns = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
                       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                      'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 
                      'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                       'MasVnrType',
                       'Foundation', 'BsmtFinType1', 'BsmtFinType2',
                       'Heating', 'CentralAir', 'Electrical', 'Functional', 
                       'GarageType', 'GarageFinish', 'PavedDrive', 'Fence',
                       'MiscFeature', 'SaleType', 'SaleCondition']
drops = ['GarageArea', 'GarageYrBlt', 'Id', 'FullBath',
         'TotRmsAbvGrd', 'X2ndFlrSF', 'BsmtFullBath',
         'MSZoning', 'SalePrice', ]
categorical_columns = list(filter(lambda v: v not in drops, categorical_columns))

In [43]:
df = pd.read_csv('post_process_train.csv', index_col = 0)

y = list(df['SalePrice'])
df = df.drop(drops, axis = 1)
pd.set_option('display.max_columns', None)
df_new = pd.concat([pd.DataFrame(fs.VarianceThreshold(threshold = .1).fit_transform(df.drop(categorical_columns, axis=1))), df[categorical_columns]], axis=1)
df_new = df_new.drop(0, axis = 0)
del y[0]
df_new = df_new.drop(len(df), axis = 0)
df_new

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,MSSubClass,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,BsmtFinType1,BsmtFinType2,Heating,CentralAir,Electrical,Functional,GarageType,GarageFinish,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition
1,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,4.0,3.0,4.0,3.0,2.0,486.0,0.0,434.0,920.0,5.0,920.0,0.0,1786.0,1.0,3.0,4.0,1.0,3.0,2.0,3.0,3.0,0.0,42.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,2002.0,60.0,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,None,None,WD,Normal
2,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,3.0,3.0,3.0,4.0,1.0,216.0,0.0,540.0,756.0,4.0,961.0,0.0,1717.0,0.0,3.0,4.0,1.0,4.0,3.0,3.0,3.0,0.0,35.0,272.0,0.0,0.0,0.0,0.0,2.0,2006.0,1970.0,60.0,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,None,None,WD,Normal
3,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,4.0,3.0,4.0,3.0,3.0,655.0,0.0,490.0,1145.0,5.0,1145.0,0.0,2198.0,1.0,4.0,4.0,1.0,3.0,3.0,3.0,3.0,192.0,84.0,0.0,0.0,0.0,0.0,0.0,12.0,2008.0,2000.0,70.0,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,BrkTil,ALQ,Unf,GasA,Y,SBrkr,Typ,Detchd,Unf,Y,None,None,WD,Abnorml
4,85.0,14115.0,5.0,5.0,1993.0,1995.0,0.0,3.0,3.0,4.0,3.0,1.0,732.0,0.0,64.0,796.0,5.0,796.0,0.0,1362.0,1.0,1.0,3.0,0.0,0.0,2.0,3.0,3.0,40.0,30.0,0.0,320.0,0.0,0.0,700.0,10.0,2009.0,1995.0,60.0,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,None,None,WD,Normal
5,75.0,10084.0,8.0,5.0,2004.0,2005.0,186.0,4.0,3.0,5.0,3.0,3.0,1369.0,0.0,317.0,1686.0,5.0,1694.0,0.0,1694.0,0.0,3.0,4.0,1.0,4.0,2.0,3.0,3.0,255.0,57.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,2005.0,50.0,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,Gable,CompShg,VinylSd,VinylSd,None,Wood,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,Unf,Y,MnPrv,Shed,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405,62.0,7917.0,6.0,5.0,1999.0,2000.0,0.0,3.0,3.0,4.0,3.0,1.0,0.0,0.0,953.0,953.0,5.0,953.0,0.0,1647.0,1.0,3.0,3.0,1.0,3.0,2.0,3.0,3.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,8.0,2007.0,2000.0,20.0,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,PConc,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,None,None,WD,Normal
1406,85.0,13175.0,6.0,6.0,1978.0,1988.0,119.0,3.0,3.0,4.0,3.0,1.0,790.0,163.0,589.0,1542.0,3.0,2073.0,0.0,2073.0,0.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,349.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2010.0,1988.0,60.0,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,PConc,Unf,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,None,None,WD,Normal
1407,66.0,9042.0,7.0,9.0,1941.0,2006.0,0.0,5.0,4.0,3.0,4.0,1.0,275.0,0.0,877.0,1152.0,5.0,1188.0,0.0,2340.0,0.0,4.0,4.0,2.0,4.0,1.0,3.0,3.0,0.0,60.0,0.0,0.0,0.0,0.0,2500.0,5.0,2010.0,2006.0,20.0,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,Gable,CompShg,Plywood,Plywood,Stone,CBlock,ALQ,Rec,GasA,Y,SBrkr,Min1,Attchd,Unf,Y,MnPrv,None,WD,Normal
1408,68.0,9717.0,5.0,6.0,1950.0,1996.0,0.0,3.0,3.0,3.0,3.0,2.0,49.0,1029.0,0.0,1078.0,4.0,1078.0,0.0,1078.0,0.0,2.0,4.0,0.0,0.0,1.0,3.0,3.0,366.0,0.0,112.0,0.0,0.0,0.0,0.0,4.0,2010.0,1996.0,70.0,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,CemntBd,CmentBd,None,Stone,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,GdPrv,Shed,WD,Normal


# Box Cox Transformation

In [ ]:
y = scipy.stats.boxcox(y, .25)
pd.set_option('display.max_columns', None)
df.head()

# Read in Kaggle Data

In [ ]:
df_kaggle = pd.read_csv('post_process_test.csv', index_col = 0).dropna()
del drops [:-1]
df_kaggle = df_kaggle.drop(drops, axis = 1)
df_kaggle = pd.get_dummies(data=df_new,columns = categorical_columns)

In [ ]:
len(df_kaggle.columns)

# Train/Test Split

In [44]:
df_dum = pd.get_dummies(data=df_new,columns = categorical_columns)
df_dum.shape

(1409, 255)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(df_dum, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
df_kaggle.shape

##AIC & BIC

In [ ]:
X_add_const = sm.add_constant(X_train)
ols = sm.OLS(y_train, X_add_const)
ans = ols.fit()
print(ans.summary())

#Multiple Linear Regression

In [46]:
#sc = StandardScaler().fit(X_train)
#X_train = sc.transform(X_train)
#X_test = sc.transform(X_test)

mlr = linear_model.LinearRegression()
mlr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [47]:
mlr.score(X_train, y_train)

0.9080591039409477

In [48]:
mlr.score(X_test, y_test)

0.8348560230976678

In [103]:
mlr_mae = metrics.mean_absolute_error(y_test, mlr.predict(X_test))
mlr_mae

20570.260790088232

In [ ]:
#Lasso Regression

In [117]:
lasso = Lasso()
lasso.fit(X_train, y_train)

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 223567832197.5207, tolerance: 576289559.4823219
  positive)


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [118]:
lasso.score(X_train, y_train)

0.9080384446825596

In [119]:
lasso.score(X_test, y_test)

0.8355768598870899

In [120]:
lasso_mae = metrics.mean_absolute_error(y_test, lasso.predict(X_test))
lasso_mae

20502.371519223943

# Random Forest

In [95]:
from sklearn.ensemble import RandomForestRegressor

#rf = RandomForestRegressor(n_estimators = 100)
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [96]:
rf.score(X_train, y_train)

0.9843168518467397

In [97]:
rf.score(X_test, y_test)

0.8614940539177454

In [108]:
rf_mae = metrics.mean_absolute_error(y_test, rf.predict(X_test))
rf_mae

16556.458758865247

# Beginning of XGBoost

In [81]:
mod = xgb.XGBRegressor(
    #gamma=1,                 
    #learning_rate=0.01,
    #max_depth=3,
    #n_estimators=10000,                                                                    
    #subsample=0.8,
    random_state=34
) 

mod.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=34, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [82]:
mod.score(X_train,y_train)

0.9997299266935789

In [83]:
mod.score(X_test,y_test)

0.8640302714303044

In [110]:
xgb_mae = metrics.mean_absolute_error(y_test, mod.predict(X_test))
xgb_mae

17107.935477615247

# SVR

In [58]:
svr = SVR()
svr.fit(X_train,y_train)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [59]:
svr.score(X_train,y_train)

-0.047850488553410786

In [60]:
svr.score(X_test,y_test)

-0.03678990769936141

# Models w/ Grid Search

In [144]:
mlrCV = linear_model.LinearRegression()
mlr_grid_param = [{'copy_X':[True,False], 'fit_intercept': [True,False], 'n_jobs': ['None'], 'normalize':[True,False]}]
para_search = GridSearchCV(estimator=mlrCV, param_grid=mlr_grid_param, scoring='r2', cv=5, return_train_score=True)
para_search = para_search.fit(X_train, y_train)

In [145]:
print(list(para_search.cv_results_.keys()))
para_search.cv_results_
print(para_search.best_score_)
print(para_search.best_params_)
para_search.best_estimator_

['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_copy_X', 'param_fit_intercept', 'param_n_jobs', 'param_normalize', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'mean_train_score', 'std_train_score']
0.84614475785268
{'copy_X': True, 'fit_intercept': True, 'n_jobs': 'None', 'normalize': False}


LinearRegression(copy_X=True, fit_intercept=True, n_jobs='None',
                 normalize=False)

In [146]:
mlr_best = para_search.best_estimator_
mlr_best.fit(X_train, y_train)
mlr_best.score(X_train, y_train)

0.9080591039409477

In [147]:
mlr_best.score(X_test, y_test)

0.8348560230976678

In [148]:
mlr_best_mae = metrics.mean_absolute_error(y_test, mlr_best.predict(X_test))
mlr_best_mae

20570.260790088232

In [114]:
lasso_best = LassoCV(alphas=np.linspace(0.00001,1,1000), cv=10,tol = 0.00001,max_iter = 10000)

In [115]:
L=lasso_best.fit(X_train, y_train)
print('train:',lasso.score(X_train, y_train),'  test:',lasso.score(X_test, y_test))

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 196725314829.36188, tolerance: 50863565.07850293
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 196723587864.43997, tolerance: 50863565.07850293
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 196721861021.74496, tolerance: 50863565.07850293
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You m

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 196674508134.00113, tolerance: 53310912.93959524
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 196672791491.93896, tolerance: 53310912.93959524
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 196671075305.2858, tolerance: 53310912.93959524
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You mi

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 185029466274.85175, tolerance: 50585673.179269895
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 185027690673.87766, tolerance: 50585673.179269895
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 185025915315.10068, tolerance: 50585673.179269895
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. Yo

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 203216583661.95068, tolerance: 53397332.23071488
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 203214851338.67767, tolerance: 53397332.23071488
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 203213118999.79526, tolerance: 53397332.23071488
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You m

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 198660960884.52335, tolerance: 51805262.14021334
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 198659297684.6467, tolerance: 51805262.14021334
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 198657634707.58035, tolerance: 51805262.14021334
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You mi

...

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 206120878649.70795, tolerance: 52712111.33891964
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 206119296288.06067, tolerance: 52712111.33891964
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 206117714162.28934, tolerance: 52712111.33891964
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You m

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 196531666625.63907, tolerance: 50784829.93399915
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 196529892018.4977, tolerance: 50784829.93399915
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 196528117705.38278, tolerance: 50784829.93399915
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You mi

C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 193051262763.91495, tolerance: 51558529.22462539
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 193049613668.88235, tolerance: 51558529.22462539
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 193047964768.21747, tolerance: 51558529.22462539
  tol, rng, random, positive)
C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:472: ConvergenceWarning: Objective did not converge. You m

train: 0.9080384600855046   test: 0.8355782383101199


C:\Users\nates\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 223851888536.708, tolerance: 57628955.94823218
  positive)


In [116]:
lasso_best_mae = metrics.mean_absolute_error(y_test, lasso_best.predict(X_test))
lasso_best_mae

20502.25581183185

# Grid Search Random Forest

In [122]:
grid_para_tree = [{
    "criterion": ["mse"],
    "max_depth": np.linspace(start = 1, stop = 15, num = 4, dtype = int ),
    "min_samples_split": np.linspace(start=2, stop=30, num=5, dtype=int),
    "n_estimators": np.linspace(start = 80, stop = 120, num = 8, dtype = int )
}]
#tree_model.set_params(random_state=34)
grid_search_rf = GridSearchCV(rf, grid_para_tree, cv=5, scoring='r2', n_jobs=-1)
grid_search_rf = para_search.fit(X_train, y_train)

In [123]:
grid_search_rf.score(X_train, y_train)
#Make your grid coarse
#Run your first parameter, then creep up the workload
#Bayesian Hyperparameter Tuning

0.9080591039409477

In [124]:
grid_search_rf.score(X_test, y_test)

0.8348560230976678

In [125]:
rf_best_mae = metrics.mean_absolute_error(y_test, grid_search_rf.predict(X_test))
rf_best_mae

20570.260790088232

# Now XG Boost w/ CV

In [129]:
#Inspiration for this approach from https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

In [130]:
cv_params = xgb.cv(
    params,
    dtrain,
    num_boost_round=999,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_params


[14:22:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,125138.326563,1078.742225,125353.090625,4721.924686
1,88378.856250,778.723453,88611.865625,3558.079023
2,62562.446094,565.039877,63252.411719,2785.816612
3,44372.863281,429.569311,46048.371875,2283.552664
4,31729.616797,283.775391,34624.373828,1545.833123
5,22960.808594,250.469364,27506.219141,1320.132565
6,17074.887500,249.079909,23076.301562,1091.254156
7,13191.478906,199.720412,20593.310156,1023.148764
8,10716.180469,221.823812,19113.038672,862.710450
9,9078.620703,197.908032,18320.957422,850.567256


In [131]:
min_mae = float("Inf")
best_params = None
for max_depth in range(9,12):
    params['max_depth'] = max_depth
    # Run CV
    cv_params = xgb.cv(
        params,
        dtrain,
        num_boost_round=999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_params['test-mae-mean'].min()
    boost_rounds = cv_params['test-mae-mean'].argmin()
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth)
print("Best params: {}, MAE: {}".format(best_params, min_mae))

[14:22:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:17

In [132]:
params['max_depth'] = best_params

In [133]:
min_mae = float("Inf")
best_params = None
for min_child_weight in range(3,15):
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_params = xgb.cv(
        params,
        dtrain,
        num_boost_round=999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_params['test-mae-mean'].min()
    boost_rounds = cv_params['test-mae-mean'].argmin()
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (min_child_weight)
print("Best params: {}, MAE: {}".format(best_params, min_mae))

[14:22:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:17

[14:22:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:17

In [134]:
params['min_child_weight'] = best_params

In [135]:
min_mae = float("Inf")
best_params = None
for subsample_bytree in range(4,10):
    subsample_bytree = subsample_bytree/10.0
    print(subsample_bytree)
    params['subsample_bytree'] = subsample_bytree
    
   
    # Run CV
    cv_params = xgb.cv(
        params,
        dtrain,
        num_boost_round=999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
     #Update best MAE
    mean_mae = cv_params['test-mae-mean'].min()
    boost_rounds = cv_params['test-mae-mean'].argmin()
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample_bytree)
print("Best params: {}, MAE: {}".format(best_params, min_mae))

0.4
[14:22:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:58] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:22:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:22:58] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  pa

0.7
[14:23:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:03] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:23:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:03] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  pa

Best params: 0.4, MAE: 16660.1591798


In [137]:
params['subsample_bytree'] = best_params


In [138]:
min_mae = float("Inf")
best_params = None
for colsample in range(3,10):
    colsample = colsample/10.
    params['colsample'] = colsample
    # Run CV
    cv_params = xgb.cv(
        params,
        dtrain,
        num_boost_round=999,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_params['test-mae-mean'].min()
    boost_rounds = cv_params['test-mae-mean'].argmin()
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (colsample)
print("Best params: {}, MAE: {}".format(best_params, min_mae))

[14:23:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:14] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:23:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:14] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language

[14:23:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:23:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language

[14:23:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:24] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[14:23:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.1.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:23:24] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample, subsample_bytree } might not be used.

  This may not be accurate due to some parameters are only used in language

In [139]:
params['colsample'] = best_params
params

{'max_depth': 9,
 'min_child_weight': 12,
 'subsample': 1,
 'colsample_bytree': 1,
 'objective': 'reg:linear',
 'subsample_bytree': 0.4,
 'colsample': 0.3}

In [140]:
mod_best = xgb.XGBRegressor(
    max_depth=params['max_depth'],                 
    min_child_weight=params['min_child_weight'],
    subsample = params['subsample'],
    colsample = params['colsample_bytree'],
    n_estimators=10000                                                                    
    
) 

In [141]:
mod_best.fit(X_train, y_train)
mod_best.score(X_train,y_train)

[14:23:29] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { colsample } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




0.9999999999997797

In [142]:
mod_best.score(X_test,y_test)

0.8688921943884749

In [143]:
mod_best_mae = metrics.mean_absolute_error(y_test, mod_best.predict(X_test))
mod_best_mae

16874.399656471633

# Undo Box Cox Transformation

In [ ]:
y_test = scipy.special.inv_boxcox(y_test, 0.25)
#Undo both or y test only

In [ ]:
mod.score(X_test,y_test)

In [ ]:
mod_best.score(X_test,y_test)